In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f
import warnings
warnings.filterwarnings('ignore')

spark = SparkSession.builder.master('local[*]').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/02 06:46:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/02 06:46:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/02 06:46:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
path_estabelecimento = '/home/rafael/Downloads/estabelecimentos/'
estabelecimento = spark.read.csv(path_estabelecimento, sep=';', inferSchema=True)


path_empresas = '/home/rafael/Downloads/socios/'
empresas = spark.read.csv(path_empresas, sep=';', inferSchema=True)

In [3]:
estabsColNames = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']

empresasColNames = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

In [4]:
for index, colName in enumerate(empresasColNames):
    empresas = empresas.withColumnRenamed(f"_c{index}", colName)

In [5]:
for index, colName in enumerate(estabsColNames):
    estabelecimento = estabelecimento.withColumnRenamed(f"_c{index}", colName)

In [6]:
empresas = empresas.withColumn('qualificacao_do_socio', f.regexp_replace('qualificacao_do_socio',',','.'))
empresas = empresas.withColumn('qualificacao_do_socio', empresas['qualificacao_do_socio'].cast(DoubleType()))
empresas = empresas.withColumn("data_de_entrada_sociedade", f.to_date(empresas.data_de_entrada_sociedade.cast(StringType()),'yyyyMMdd'))

In [7]:
empresas.createOrReplaceTempView('empresasView')

In [9]:
spark.sql('SELECT * FROM empresasView').show(5)

+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social|cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|        411|                     2|         LILIANA PATRICIA ...|         ***678188**|                 22.0|               1994-07-25|null|        ***000000**|                 null|                                  0|           7|
|        411|                     2|         CRISTINA HUNDERTMARK|      

In [11]:
spark\
    .sql("""
        SELECT * 
            FROM empresasView 
            WHERE faixa_etaria >3
    """)\
    .show(5)

+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social|cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|        411|                     2|         LILIANA PATRICIA ...|         ***678188**|                 22.0|               1994-07-25|null|        ***000000**|                 null|                                  0|           7|
|        411|                     2|         CRISTINA HUNDERTMARK|      

In [12]:
spark\
    .sql("""
        SELECT qualificacao_do_socio, MEAN(faixa_etaria) AS Media 
            FROM empresasView 
            GROUP BY qualificacao_do_socio
    """)\
    .show(5)

+---------------------+-----------------+
|qualificacao_do_socio|            Media|
+---------------------+-----------------+
|                  8.0| 5.71825764596849|
|                 49.0|5.494259246467019|
|                 29.0|6.221374045801527|
|                 18.0|6.623809523809523|
|                 37.0|              0.0|
+---------------------+-----------------+
only showing top 5 rows

